Дать выбрать персонажа из списка типичных или написать самому (лемматизировать)
<br>
Набрать корпус анекдотов по сайтам краулингом, искать по словоформам, мб векторы??, чекни, что повторов нет,  dataset записать

Предпроцессинг: наверное, чекнуть на имена собственные (большие буквы не после !.?, м.б. чекнуть Наташей, можно ли это сделать быстрее, чем in,  выкинуть остальное)

Дать выбирать (не)приличные, могу ли я автоматически так разметить? маш обуч??

В презентации объяснить, почему именно такие сайты

**Сайты:**
<br>
https://www.anekdot.ru/release/anekdot/day/2020-02-06/
<br>
https://anekdoty.ru/
<br>
https://archive.org/download/120_tysyach_anekdotov
<br>
file:///C:/Users/obrid/Downloads/Nikulin_Anekdoty_ot_Nikulina.pdf


**Дизайн сайта**

Две страницы:


*   На первой город, откуда человек, его пол. Выбрать персонажа, которого ищем в анекдоте из списка ниже или напечатать самому. Выбрать, только ли приличные анекдоты. Ссылка - оглавление сверху, ссылки на источники анекдотов
*   На второй выводим анекдот, статистику столбчатую по женщинам и мужчинам, каких персонажей выбирали (топ 6 персонажей типа)
*   Прикольный дизайн!!

<br>
**Порядок действий:**


1. Проверь, подошли ли сайты. Считай данные по ним и анекдоты из Никулина в датасеты (три разных), только те, где есть имена собственные или определённые слова
# New section
2.   Разметь где-то 800 на приличность, обучи машинку, проверь на 200, запусти разметку на всё
3.   Сделай сайтик, сделай функции
4. Загрузить всё на гитхаб, pythoneverywhere
5. Сверься с критериями
5. Ура, всё получилось)

<br>

**Презентация:**

- объясни, почему именно такие источники для анекдотов
- расскажи про этапы деланья, особенно про обучение модели
- сделай прикольную презу
- та да






**Про кого анекдоты отдельно смотрим:**
- Ленин

- Сталин
- Хрущев
- Брежнев
- Горбачев
- Ельцин
- Путин
- Петр Первый
- Медведев
- Вовочка
- Гаишник
- Теща
- Чукча
- Новый русский
- Блондинка
- американец, француз, русский (другой набор национальностей)
- грузин, хохол, горячий эстонский парень
- Еврей
- Штирлиц

In [ ]:
import spacy
!python -m spacy download ru_core_news_sm
nlp = spacy.load("ru_core_news_sm")
import ru_core_news_sm


In [ ]:
import sqlite3
conn = sqlite3.connect('anekdotes.db')
c = conn.cursor()

In [ ]:
def space(line):
  global bad
  bad = 0
  doc = nlp(line)
  for ent in doc.ents:
    if (ent.label_ == "PER"):
      bad += 1
  for i in doc:
    if i.lemma_ in ["жена", "теща", "блондинка", "американец", "француз", "русский", "грузин", "хохол", "эстонец", 'еврей', "чукча", "одессит" ]:
      bad += 1
  return

In [ ]:
def Nikulin():
  global bad
  drop = []
  with open("Nikulin.txt", "r+", encoding="utf-8") as Nik:
    #подумай, как совсем очистить от надписей про книгу
    lines = [i for i in Nik.readlines() if ("100 лучших книг всех времен: www.100bestbooks.ru" not in i) or ("Юрий Никулин «Анекдоты от Никулина»" not in i)]
    lines = "".join(lines)
    aneks = lines.split("***\n")
    id = 0
    for i in aneks:
      space(i)
      if bad != 0:
        c.execute("CREATE TABLE IF NOT EXISTS Nikulin (id_n integer, text_n text)")
        record = (id, i)
        c.execute("INSERT INTO Nikulin VALUES (?, ?)", record)
        conn.commit()
        id += 1

In [ ]:
Nikulin()

In [ ]:
import requests
!pip install fake_useragent
from requests.exceptions import MissingSchema
from bs4 import BeautifulSoup
import time
import random
from fake_useragent import UserAgent
ua = UserAgent()
session = requests.session()

In [ ]:
from datetime import date, timedelta

In [ ]:
def anek1():
  def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

  start_date = date(2020, 2, 1)
  end_date = date(2020, 2, 8)
  dates = []
  textik = []
  id = 0
  for single_date in daterange(start_date, end_date):
    dates.append(single_date.strftime("%Y-%m-%d"))

  for i in dates:
    resp = session.get(f'https://www.anekdot.ru/release/anekdot/day/{i}/', headers={'User-Agent': ua.random})
    time.sleep(random.uniform(1.1, 5.0))
    page = resp.text
    soup = BeautifulSoup(page, "html.parser")
    textik.extend(soup.find_all("div", {"class":"text"}))
    for i in range(len(textik)):
      try:
        textik[i] = textik[i].text
      except:
        pass
      space(textik[i])
      if bad != 0:
        c.execute("CREATE TABLE IF NOT EXISTS Anek1 (id_1 integer, text_1 text)")
        record = (id, textik[i])
        c.execute("INSERT INTO Anek1 VALUES (?, ?)", record)
        conn.commit()
        id += 1

In [ ]:
anek1()

In [ ]:
def anek_2():
  namings = ["pro-evreev", 'pro-shtirlica', 'pro-vovochku', 'pro-zhenu', 'pro-armyan', 'pro-gruzinov']
  textok = []
  id = 0
  for i in namings:
    for j in range (2, 7):
      resp = session.get(f'https://anekdoty.ru/{i}/{j}/', headers={'User-Agent': ua.random})
      time.sleep(random.uniform(1.1, 5.0))
      page = resp.text
      soup = BeautifulSoup(page, "html.parser")
      textok.extend(soup.find_all("div", {"class":"holder-body"}))
  for i in range(1, 4):
      if i == 1:
        resp = session.get(f'https://anekdoty.ru/pro-gitlera/', headers={'User-Agent': ua.random})
      else:
        resp = session.get(f'https://anekdoty.ru/pro-gitlera/{i}/', headers={'User-Agent': ua.random})
      time.sleep(random.uniform(1.1, 5.0))
      page = resp.text
      soup = BeautifulSoup(page, "html.parser")
      textok.extend(soup.find_all("div", {"class":"holder-body"}))
      for i in range(len(textok)):
        try:
          textok[i] = textok[i].text
        except:
          pass
        space(textok[i])
        if bad != 0:
          c.execute("CREATE TABLE IF NOT EXISTS Anek2 (id_2 integer, text_2 text)")
          record = (id, textok[i])
          c.execute("INSERT INTO Anek2 VALUES (?, ?)", record)
          conn.commit()
          id += 1




In [ ]:
anek_2()

In [ ]:
import pandas as pd

In [ ]:
def deti():
  global de
  te = []
  for j in range (2, 12):
      resp = session.get(f'https://anekdoty.ru/detskie/{j}/', headers={'User-Agent': ua.random})
      time.sleep(random.uniform(1.1, 5.0))
      page = resp.text
      soup = BeautifulSoup(page, "html.parser")
      te.extend(soup.find_all("div", {"class":"holder-body"}))
  for i in range(len(te)):
        try:
          te[i] = te[i].text
        except:
          pass
  pril = []
  for i in range(len(te)):
    pril.append(0)
  de = pd.DataFrame({"deti": te, "pril": pril})
  return

In [ ]:
deti()

In [ ]:
def child():
  global ch
  text = ""
  pril = []
  resp = session.get(f'https://proza.ru/2013/01/20/18', headers={'User-Agent': ua.random})
  page = resp.text
  soup = BeautifulSoup(page, "html.parser")
  text = soup.find("div", {"class":"text"})
  te = str(text).replace("\n"," ").replace("<br/>"," ").split("***")
  for i in range(200):
    pril.append(0)
  ch = pd.DataFrame({"deti": te[3:203], "pril": pril})

In [ ]:
child()

In [ ]:
ch.head(5)

,deti,pril
0,Доктор: - Вы должны принять эту таблетку ...,0
1,"- Вы увольняете ценных сотрудников, кто же о...",0
2,"Две подруги: - Эдик мне вчера сказал, что,...",0
3,Иногда мои случайные знакомые удивляются том...,0
4,Один мэр города загадал на Новый Год потерят...,0


In [ ]:
def ervax():
  global er
  te = []
  for j in range (1, 5):
      resp = session.get(f'https://www.anekdot.ru/tags/%D0%BF%D0%BE%D1%88%D0%BB%D1%8B%D0%B5/{j}', headers={'User-Agent': ua.random})
      time.sleep(random.uniform(1.1, 5.0))
      page = resp.text
      soup = BeautifulSoup(page, "html.parser")
      te.extend(soup.find_all("div", {"class":"text"}))
  for i in range(len(te)):
        try:
          te[i] = te[i].text
        except:
          pass
  pril = []
  for i in range(len(te)):
    pril.append(1)
  er = pd.DataFrame({"ervax": te, "pril": pril})
  return

In [ ]:
ervax()

In [ ]:
def adult():
  global ad
  query = """
  SELECT text_1, Pril
  FROM Anek1
  LIMIT 200
  """
  ad = pd.read_sql_query(query, con=conn)
  return

In [ ]:
adult()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
d = list(ch["deti"].values)
e = list(er["ervax"].values)
a = list(ad["text_1"].values)
X = d + e + a
ed = list(ch["pril"].values)
r = list(er["pril"].values)
da = list(ad["Pril"].values)
y = ed + r + da

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
vectorizer = CountVectorizer()
X_train_transformed = vectorizer.fit_transform(X_train)
X_test_transformed = vectorizer.transform(X_test)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
tree = DecisionTreeClassifier(
    max_depth=None,
    min_samples_split=4,
    min_samples_leaf=1,
)

In [ ]:
tree.fit(X_train_transformed, y_train)
y_pred = tree.predict(X_test_transformed)
print(accuracy_score(y_test, y_pred), recall_score(y_test, y_pred, pos_label=1),
      precision_score(y_test, y_pred, pos_label=1))

0.725 0.38461538461538464 0.625


In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score

In [ ]:
print(accuracy_score(y_test, y_pred))
print(recall_score(y_test, y_pred, pos_label=1))
print(precision_score(y_test, y_pred, pos_label=1))

0.625
0.1282051282051282
0.3125


In [ ]:
def cenz1():
  query = """
  SELECT text
  FROM Anek11
  """
  df = pd.read_sql_query(query, con=conn)
  c1 = list(df['text'].values)
  c1_t = vectorizer.transform(c1)
  y1 = classifier.predict(c1_t)
  for i in range(len(y1)):
    c.execute("UPDATE Anek11 SET pril = ? WHERE id_11 = ?", (int(y1[i]), i))
    conn.commit()


In [ ]:
cenz1()

In [ ]:
def cenz2():
  query = """
  SELECT text_22
  FROM Anek22
  """
  df = pd.read_sql_query(query, con=conn)
  c1 = list(df['text_22'].values)
  c1_t = vectorizer.transform(c1)
  y1 = classifier.predict(c1_t)
  for i in range(len(y1)):
    c.execute("UPDATE Anek22 SET pril_22 = ? WHERE id_22 = ?", (int(y1[i]), i))
    conn.commit()


In [ ]:
cenz2()

In [ ]:
def cenzn():
  query = """
  SELECT text_n
  FROM Nikulin
  """
  df = pd.read_sql_query(query, con=conn)
  c1 = list(df['text_n'].values)
  c1_t = vectorizer.transform(c1)
  y1 = classifier.predict(c1_t)
  for i in range(len(y1)):
    c.execute("UPDATE Nikulin SET pril_n = ? WHERE id_n = ?", (int(y1[i]), i))
    conn.commit()

In [ ]:
cenzn()

In [ ]:
import matplotlib
import pandas as pd

from flask import Flask
from flask import render_template
from flask import request
import matplotlib.pyplot as plt



In [ ]:
 !pip install pymorphy2


In [ ]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [ ]:
def search(word):
  global final
  # word = request.args['T2']
  # apro = request.args['defaultCheck1']
  apro = 0
  word = morph.parse(word)[0]
  lex = word.lexeme
  l = []
  for i in lex:
    l.append(i.word)
  if apro == 1:
      query = """
      SELECT text_22
      FROM Anek22
      WHERE pril_22 = 0
      """
      df0 = pd.read_sql_query(query, con=conn)
      query1 = """
      SELECT text
      FROM Anek11
      WHERE pril = 0
      """
      df1 = pd.read_sql_query(query1, con=conn)
      query2 = """
      SELECT text_n
      FROM Nikulin
      WHERE pril_n = 0
      """
      df2 = pd.read_sql_query(query2, con=conn)
      data = [df0, df1, df2]
      df = pd.concat(data)
      final = []
      r = ["text_22"]
      for j in r:
        for q in list(df[j]):
          for k in l:
            if (k+ " ")  in str(q) and str(q) not in final:
              final.append(str(q))
  else:
      query = """
      SELECT text_22
      FROM Anek22
      """
      df0 = pd.read_sql_query(query, con=conn)
      query1 = """
      SELECT text
      FROM Anek11
      """
      df1 = pd.read_sql_query(query1, con=conn)
      query2 = """
      SELECT text_n
      FROM Nikulin
      """
      df2 = pd.read_sql_query(query2, con=conn)
      data = [df0, df1, df2]
      df = pd.concat(data)
      final = []
      r = ["text_22"]
      for j in r:
        for q in list(df[j]):
          for k in l:
            if (k+ " ")  in str(q) and str(q) not in final:
              final.append(str(q))
  return


In [ ]:
search("отец")

In [ ]:
print(final)

['— Штирлиц, — сказал Мюллер, — вы, часом, не еврей?— Ну, да! Мать русская, отец русский, а я почему—то еврей, — обиделся Штирлиц и подумал: — Не сболтнул ли я чего лишнего?', 'Приехал Вовочка с отцом в лес. Сидят, жарят сосиски, разговаривают.— Папа, я хочу срать.— Не стесняйся! Мы в лесу — сри где хочешь и ничего тебе за это не будет!Через некоторое время Вовочка возвращается.— Ну что, где ты это сделал?— В твоей машине.', '— Вовочка, как случилось так, что Машенька забеременела?— А—а—а, ее отец виноват.— ???— Я только дал назад, как он вошел в комнату и ка—а—а—ак шлепнул меня по заднице...', 'Вовочка не выучил урок, учительница собирается ставить двойку. — Марья Ивановна, я не хочу вас пугать, но мне отец сказал: ”Еще одна двойка — и кто—то получит по голове!”', 'Марья Ивановна пригласила отца Вовочки:— Ваш сын назвал меня оху*вшей тварью и отказался выйти к доске!— Сынок, если эта оху*вшая тварь приглашает к доске, то надо идти!', 'Марья Ивановна возвращает Вовочке дневник и говори